# BLEU Score(Bilingual Evaluation Understudy Score)

앞서 언어 모델의 성능 측정을 위한 평가 방법으로 Perplexity(PPL)를 배운적이 있습니다. 기계 번역기에도 PPL을 평가에 사용할 수는 있지만, PPL은 번역의 성능을 직접적으로 반영하는 수치라 보기엔 어렵습니다.

자연어 처리에서는 그 외에도 수많은 평가 방법들이 존재하는데, 기계 번역의 성능이 얼마나 뛰어난가를 측정하기 위해 사용되는 대표적인 방법인 BLEU(Bilingual Evaluation Understury)에 대해서 학습해보겠습니다.

## 1. BLEU(Bilingual Evaluation Understudy)

BLEU는 기계 번역 결과와 사람이 직접 번역한 결과가 얼마나 유사한지 비교하여 번역에 대한 성능을 측정하는 방법입니다. 측정 기준은 n-gram에 기반합니다.

BLEU는 언어에 구애받지 않고 사용할 수 있으며, 계산 속도가 빠르다는 장점이 있습니다. BLEU는 PPL과 달리 높을 수록 성능이 더 좋음을 의미합니다. 
<br />

### 1) 단어 개수 카운트로 측정하기(Unigram Precision)

한국어-영어 번역기의 성능을 측정한다고 가정해봅시다. 두 개의 기계 번역기가 존재하고 두 기계 번역기에 같은 한국어 문장을 입력하여 번역된 영어 문장의 성능을 측정하고자 합니다.

번역된 문장을 각각 Candidate 1, 2라고 해봅시다. 이 문장의 성능을 평가하기 위해서는 정답으로 비교되는 문장이 있어야 합니다.

세 명의 사람에게 한국어를 보고 영작해보라고 하여 세 개의 번역 문장을 만들어냈습니다. 이 세 문장을 각각 Reference 1, 2, 3이라고 해보겠습니다.
<br />
<br />
- Candidate1 : It is a guide to action which ensures that the military always obeys the commands of the party.

- Candidate2 : It is to insure the troops forever hearing the activity guidebook that party direct.

- Reference1 : It is a guide to action that ensures that the military will forever heed Party commands.

- Reference2 : It is the guiding principle which guarantees the military forces always being under the command of the Party.

- Reference3 : It is the practical guide for the army always to heed the directions of the party.
<br />
<br />

편의상 Candidate를 Ca로, Reference를 Ref로 축약하여 부르겠습니다.

가장 직관적인 성능 평가 방법은 Ref 1, 2, 3중 어느 한 문장이라도 등장한 단어의 개수를 Ca에서 세는 것입니다. 그리고 그 후에 Ca의 모든 단어의 카운트의 합. 즉, Ca에서의 총 단어의 수로 나눠줍니다.
<br />
<br />

$$\text{Unigram Precision} = \frac{\text{Ref들 중에서 존재하는 Ca의 단어의 수}}{{\text{Ca의 총 단어의 수}}}$$
<br />
<br />
Ca1의 단어들은 얼추 훑어만 봐도 Ref 1, 2, 3에 전반적으로 등장하는 반면, Ca2는 그렇지 않습니다. 이는 Ca1이 Ca2보다 더 좋은 번역 문장임을 의미합니다.<br /><br />
위의 계산 방법에 따르면 Ca1과 Ca2의 유니그램 정밀도는 각각 아래와 같습니다.<br /><br />

$$\begin{aligned} \text{Ca1 Unigram Precision} &= \frac{17}{18}\\ \text{Ca2 Unigram Precision} &= \frac{8}{14}\end{aligned}$$

이제부터 단어라는 표현보다는 유니그램이라는 용어로 설명하겠습니다. 지금까지 설명한 유니그램 정밀도는 나름 의미있는 측정 방법으로 보이지만 사실 허술한 점이 있습니다. 아래와 같은 새로운 예가 있다고 해보겠습니다.
<br /><br />

### 중복을 제거하여 보정하기(Modified Unigram Precision)

<br />

- Candidate : the the the the the the the

- Reference1 : the cat is on the mat

- Reference2 : there is a cat on the mat

<br />

위의 Ca는 the만 7개가 등장한 터무니없는 번역입니다. 하지만 이 번역은 앞서 배운 유니그램 정밀도에 따르면 $\frac{7}{7} = 1$이라는 최고의 성능 평가를 받게 됩니다. 따라서 유니그램 정밀도를 보정할 필요가 있습니다.
<br />

이를 보정하기 위해서는 정밀도의 분자를 계산하기 위해 Ref와 매칭하여 카운트하는 과정에서 Ca의 유니그램이 이미 Ref에서 매칭된 적이 있었는지를 고려해야 합니다.
<br />
<br />

$$\text{Unigram Precision} = \frac{\text{Ref들과 Ca를 고려한 새로운 카운트 방법!!!}}{\text{Ca의 총 유니그램 수}}$$

<br />
정밀도의 분자를 계산하기 위한 각 유니그램의 카운트는 다음과 같이 수정하겠습니다. 
<br />

우선, 유니그램이 하나의 Ref에서 최대 몇 번 등장했는지 카운트합니다. 이 값을 maximum reference count(Max_Ref_Count)라고 부르겠습니다.
<br />

Max_Ref_Count가 기존의 단순 카운트한 값보다 작은 경우에는 이 값을 최종 카운트 값으로 대체합니다. 정밀도의 분자 계산을 위한 새로운 카운트 방식을 식으로 표현하면 아래와 같습니다.

<br />

$\text{Count}\_{\text{clip}} = min(\text{Count},\; \text{Max\_Ref\_Count})$

<br />

위의 카운트를 사용하여 분자를 계산한 정밀도를 **보정된 유니그램 정밀도(Modified Unigram Precision)**라고 합니다.

<br />

$$\text{Modified Unigram Precision} = \frac{\text{Ca의 각 유니그램에 대해 \text{Count}\_{\text{clip}} 을 수행한 값의 총 합}}{\text{Ca의 총 유니그램 수}}$$

<br />

### 3) 보정된 유니그램 정밀도(Modified Unigram Precision) 구현하기


In [2]:
import numpy as np
from collections import Counter
from nltk import ngrams

# tokens에서 n-gram 단순 카운트
def simple_count(tokens, n):
    return Counter(ngrams(tokens, n))

In [3]:
# Candidate 1 
candidate = "It is a guide to action which ensures that the military always obeys the commands of the party."
tokens = candidate.split()
result = simple_count(tokens, 1) # n=1 유니그램
print('Unigram count : ', result)


Unigram count :  Counter({('the',): 3, ('It',): 1, ('is',): 1, ('a',): 1, ('guide',): 1, ('to',): 1, ('action',): 1, ('which',): 1, ('ensures',): 1, ('that',): 1, ('military',): 1, ('always',): 1, ('obeys',): 1, ('commands',): 1, ('of',): 1, ('party.',): 1})


In [4]:
# Candidate (Example 2)
candidate = 'the the the the the the the'
tokens = candidate.split()
result = simple_count(tokens, 1)
print('Unigram count : ', result)

Unigram count :  Counter({('the',): 7})


In [5]:
# Count_clip 
def count_clip(candidate, reference_list, n):
    # Ca 문장에서 n-gram count
    ca_cnt = simple_count(candidate, n)
    max_ref_cnt_dict = dict()

    for ref in reference_list:
        # Ref 문장에서 n-gram count
        ref_cnt = simple_count(ref, n)

        # 각 Ref 문장에 대해 비교하여 n-gram의 최대 등장 횟수를 계산
        for n_gram in ref_cnt:
            if n_gram in max_ref_cnt_dict:
                max_ref_cnt_dict[n_gram] = max(ref_cnt[n_gram], max_ref_cnt_dict[n_gram])
            else:
                max_ref_cnt_dict[n_gram] = ref_cnt[n_gram]
    
    return {
        # count_clip = min(count, max_ref_count)
        n_gram : min(ca_cnt.get(n_gram, 0), max_ref_cnt_dict.get(n_gram, 0)) for n_gram in ca_cnt
    }

In [6]:
candidate = 'the the the the the the the'
references = [
    'the cat is on the mat',
    'there is a cat on the mat'
]

result = count_clip(candidate.split(), list(map(lambda ref: ref.split(), references)), 1)
print('Modified Unigram Count : ', result)

Modified Unigram Count :  {('the',): 2}


In [8]:
# Modified Unigram Precision
def modified_precision(candidate, reference_list, n):
    clip_cnt = count_clip(candidate, reference_list, n)
    total_clip_cnt = sum(clip_cnt.values()) # 분자

    cnt = simple_count(candidate, n)
    total_cnt = sum(cnt.values()) # 분모

    # 분모 != 0
    if total_cnt == 0:
        total_cnt = 1

    # 분자 : count_clip의 합
    # 분모 : 단순 count의 합
    return (total_clip_cnt / total_cnt)

result = modified_precision(candidate.split(), list(map(lambda ref: ref.split(), references)), 1)
print('Modified Unigram Precision : ', result)

Modified Unigram Precision :  0.2857142857142857


### 4) 순서를 고려하기 위해서 n-gram으로 확장하기

<br />

BoW 표현과 유사하게, 유니그램 정밀도와 같이 각 단어의 빈도수로 접근하는 방법은 결국 단어의 순서를 고려하지 않는다는 특징이 있습니다.
<br />

Example 1에 Ca3라는 새로운 문장을 추가해보고 기존의 Ca1과 비교해보겠습니다.
<br />

- Candidate1 : It is a guide to action which ensures that the military always obeys the commands of the party.

- Candidate2 : It is to insure the troops forever hearing the activity guidebook that party direct.

- Candidate3 : the that military a is It guide ensures which to commands the of action obeys always party the.

- Reference1 : It is a guide to action that ensures that the military will forever heed Party commands.

- Reference2 : It is the guiding principle which guarantees the military forces always being under the command of the Party.

- Reference3 : It is the practical guide for the army always to heed the directions of the party.

<br />
Ca3는 사실 Ca1에서 모든 유니그램의 순서를 랜덤으로 섞은 실제 영어 문법에 맞지 않는 문장입니다. 하지만 Ref 1, 2, 3과 비교하여 유니그램 정밀도를 적용하면 Ca1과 Ca3의 정밀도는 동일합니다. 유니그램 정밀도는 유니그램의 순서를 전혀 고려하지 않기 때문입니다.
<br />

이를 위한 대안으로 유니그램 왜에도 Bigram, Trigram, 4-gram 단위 등으로 계산한 정밀도. 즉, n-gram을 이용한 정밀도를 적용하고자 합니다.
<br />

이들 각각은 카운트 단위를 2개, 3개, 4개로 보느냐의 차이입니다. 어떤 의미인지 Bigram단위로 카운트하여 정밀도를 계산해보겠습니다.
<br />

- Candidate1 : the the the the the the the

- Candidate2 : the cat the cat on the mat

- Reference1 : the cat is on the mat

- Reference2 : there is a cat on the mat
<br />

Ca2 바이그램의 Count와 $\text{Count}_{\text{clip}}$ 은 아래와 같습니다.
<br />

|바이그램|the cat|cat the|cat on|on the|the mat|SUM|
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|Count|2|1|1|1|1|6|
|$\text{Count}_{\text{clip}}$ |1|0|1|1|1|4|

<br />
결과적으로 Ca2의 바이그램 정밀도는 4/6가 됩니다. 
<br />

이처럼 n-gram을 이용하여 보정된 정밀도를 계산하는 식을 작성해보면 아래와 같습니다.
<br />

$$p_n = \frac{\sum_{\text{n-gram} \in \text{Candidate}} \text{Count}_{\text{clip}} (\text{unigram})}{\sum_{\text{n-gram} \in \text{Candidate}} \text{Count} (\text{n-gram})}$$
<br />

n-gram 정밀도 식을 이해하였다면 BLEU의 최종 식까지 다 왔습니다. BLEU는 보정된 정밀도 $p_1,\; p_2,\; p_3,\; \ldots,\; p_n$를 모두 조합하여 사용합니다. 이를 모두 조합한 BLEU의 식은 아래와 같습니다.
<br />

$$\text{BLEU} = \exp(\sum_{n=1}^N w_n \log p_n)$$
<br />

- $p_n$ : 각 n-gram의 보정된 정밀도입니다.

- $N$ : n-gram에서 n의 최대 숫자입니다. 보통은 4를 가지고, $N$이 4라는 것은 $p_1,\; p_2,\; p_3,\; p_4$를 사용한다는 것을 말합니다.

- $w_n$ : 각 n-gram의 보정된 정밀도에서 서로 다른 가중치를 줄 수 있습니다. 이 가중치의 합은 1이 되도록 합니다.
<br />

### 5) 짧은 문장 길이에 대한 패널티(Brevity Penalty)
<br />

n-gram으로 단어의 순서를 고려한다고 하더라도 여전히 남아있는 문제가 있는데, 바로 Ca의 길이에 BLEU의 점수가 과한 영향을 받을 수 있다는 점입니다. 
<br />

example 1에 다음의 Ca를 추가한다고 해보겠습니다.
<br />

- Candidate 4 : it is
<br />

이 문장은 유니그램 정밀도나 바이그램 정밀도가 각각 $\frac{2}{2},\; \frac{1}{1}$로 두 정밀도 모두 1이라는 높은 정밀도는 얻습니다. 이와 같이 제대로 된 번역이 아님에도 문장의 길이가 짧다는 이유로 높은 점수를 받는 것은 이상합니다. 그래서 Ca가 Ref보다 문장의 길이가 짧은 경우에는 점수에 패널티를 줄 필요가 있습니다. 이를 브레버티 패너티(Brevity Penalty)라고 합니다. 
<br />

이에 대해 살펴보기 전에, 만약 반대고 Ca의 길이가 Ref보다 긴 경우에도 문제가 생길 수 있는지 보겠습니다.
<br />

- Candidate 1: I always invariably perpetually do.

- Candidate 2: I always do.

- Reference 1: I always do.

- Reference 2: I invariably do.

- Reference 3: I perpetually do.
<br />

위 예시에서 Ca1은 가장 많은 단어를 사용했지만 Ca2보다 좋지 못한 번역입니다. 다시 말해 Ref의 단어를 가장 많이 사용한 것이 꼭 좋은 번역이라는 의미는 아닙니다. 하지만 다행히도 위와 같이 Ca의 길이가 불필요하게 Ref보다 긴 경우에는 BLEU 수식에서 정밀도를 n-gram으로 확장하여 바이그램, 트라이그램 정밀도 등을 모두 계산에 사용하고 있는 것만으로도 이미 패널티를 받고 있습니다. 즉, Brevity Penalty를 설계할 때 이 경우까지 고려할 필요는 없습니다.
<br />

다시 Ref보다 Ca의 길이가 짧을 경우에 패널티를 주는 Brevity Penalty의 이야기로 돌아와보겠습니다. Brevity Penalty는 앞서 배운 BLEU의 식에 곱하는 방식으로 사용합니다. Brevity Penalty를 BP라고 했을 때, 최종 BLEU식은 아래와 같습니다.
<br />

$$\text{BLEU} = \text{BP} \times \exp(\sum_{n=1}^N w_n \log p_n)$$
<br />

위 식은 패널티를 줄 필요가 없는 경우에는 BP의 값이 1이어야 함을 의미합니다. 이를 반영한 BP의 수식은 아래와 같습니다.
<br />

$$\text{BP} = \begin{cases} 1 & \text{if}\; c > r\\ e^{(1-r/c)} & \text{if}\; c \leq r\end{cases}$$
<br />

- $c$ : Candidate length

- $r$ : Candidate와 가장 길이 차이가 적은 Reference의 길이
<br />

In [9]:
## BLEU 구현

# Ca 길이와 가장 근접한 Ref의 길이를 리턴하는 함수
def closest_ref_length(candidate, reference_list):
    ca_len = len(candidate)
    ref_lens = (len(ref) for ref in reference_list)

    closest_ref_length = min(ref_lens, key=lambda ref_len : (abs(ref_len - ca_len), ref_len))
    return closest_ref_length

# Brevity Penalty
def brevity_penalty(candidate, reference_list):
    ca_len = len(candidate)
    ref_len = closest_ref_length(candidate, reference_list)

    if ca_len > ref_len:
        return 1
    
    # candidate가 비어있다면 BP = 0 -> BLEU = 0.0
    elif ca_len == 0:
        return 0
    else:
        return np.exp(1 - ref_len / ca_len)
    
# BLEU SCORE
def bleu_score(candidate, reference_list, weights=[0.25, 0.25, 0.25, 0.25]):
    bp = brevity_penalty(candidate, reference_list)

    p_n = [modified_precision(candidate, reference_list, n=n) for n, _ in enumerate(weights, start=1)]

    # p1, p2, p3, ..., pn
    score = np.sum([w_i * np.log(p_i) if p_i != 0 else 0 for w_i, p_i in zip(weights, p_n)])
    return bp * np.exp(score)

## 2. NLTK를 사용한 BLEU 측정하기

In [10]:
import nltk.translate.bleu_score as bleu

candidate = 'It is a guide to action which ensures that the military always obeys the commands of the party'
references = [
    'It is a guide to action that ensures that the military will forever heed Party commands',
    'It is the guiding principle which guarantees the military forces always being under the command of the Party',
    'It is the practical guide for the army always to heed the directions of the party'
]

print('실습 코드로 구현한 BLEU : ',bleu_score(candidate.split(), list(map(lambda ref : ref.split(), references))))
print()
print('NLTK의 BLEU : ', bleu.sentence_bleu(list(map(lambda ref: ref.split(), references)), candidate.split()))

실습 코드로 구현한 BLEU :  0.5045666840058485

NLTK의 BLEU :  0.5045666840058485


## 의문점

단순히 카운트 기반으로 점수를 측정하는데... 이러면 만약 의역이나 초월번역은 어떻게 점수를 측정할지가 궁금하다..